<a href="https://colab.research.google.com/github/HSS107048212/hoop4s/blob/main/%E5%90%88%E4%BD%9C%E7%A4%BE%E7%95%B6%E6%97%A5%E3%80%8C%E8%B3%BC%E8%B2%B7%E7%A2%BA%E8%AA%8D%E5%96%AE%E3%80%8D%E6%9F%A5%E8%A9%A2(realconfirm).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 轉換後程式

In [1]:
!pip install python-docx
# Installation and imports
!pip3 install pygsheets
!pip3 install gspread-dataframe==3.2.1
!pip3 install openpyxl
# Install required package for integrating Pandas with Google Sheets
!pip install gspread-dataframe


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 237.4/237.4 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.2/158.2 kB 3.5 MB/s eta 0:00:00
  Attempting uninstall: gspread-dataframe
    Found existing installation: gspread-dataframe 3.3.1
    Uninstalling gspread-dataframe-3.3.1:
      Successfully uninstalled gspread-dataframe-3.3.1


# SERVER 端程式

In [22]:
# Importing necessary libraries
import os
import re
from datetime import datetime

import pandas as pd
import pygsheets
import gspread
#from gspread_dataframe import set_with_dataframe
#from gspread import set_with_dataframe
from oauth2client.service_account import ServiceAccountCredentials

from docx import Document
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

import warnings
# There is a warning msg : /home/iss/.local/lib/python3.10/site-packages/gspread/worksheet.py:1069: UserWarning: [Deprecated][in version 6.0.0]: method signature will change to: 'Worksheet.update(value = [[]], range_name=)' arguments 'range_name' and 'values' will swap, values will be mandatory of type: 'list(list(...))'
# Put this filter to ignore this specific type of warning (e.g., DeprecationWarning)
warnings.filterwarnings("ignore")

# Fnction Definition
# Function to convert Timestamp to string with a specific format
def convert_to_string(timestamp):
    return timestamp.strftime('%Y-%m-%d %H:%M:%S')


#HOME_PATH = '/home/iss/realplus/'
HOME_PATH = ''
#HOME_PATH = '/content/' # path of Colab for testing

# Google Sheets Authorization
gc = pygsheets.authorize(service_file=f'{HOME_PATH}orderdata-copyto-parttime-5c19ba8fcba4.json')  # Gild學長後續請將.json金鑰設定在local路徑

# Fetching Order Data from Google Sheet
Order_SPREADSHEET = 'https://docs.google.com/spreadsheets/d/15jbMd0cvO2OCcgxZiQJLG1jSR6rXyXn1mnl7SCoFoTM/edit?usp=sharing'
sht = gc.open_by_url(Order_SPREADSHEET)
wks = sht.worksheet_by_title("表單回應 1")
Order_data = pd.DataFrame(wks.get_all_records())

# Function to convert Chinese timestamp to a suitable format
def convert_timestamp(timestamp):
    return re.sub(r'(上午|下午)', '', timestamp)

Order_data['時間戳記'] = Order_data['時間戳記'].apply(convert_timestamp)
Order_data['日期'] = pd.to_datetime(Order_data['時間戳記']).dt.strftime('%Y%m%d')
Order_data['當日訂單數量'] = Order_data.groupby('日期').cumcount().add(1).apply(lambda x: f'{x:03d}')
Order_data['訂單編號'] = Order_data['日期'] + Order_data['當日訂單數量']
Order_data.drop(['日期', '當日訂單數量'], axis=1, inplace=True)

# Selecting rows of today's date
today = datetime.today().date()
Order_data['時間戳記'] = Order_data['時間戳記'].apply(lambda x: x.split()[0])
Order_data['時間戳記'] = pd.to_datetime(Order_data['時間戳記'], format='%Y/%m/%d')
today_rows = Order_data[Order_data['時間戳記'].dt.date == today]
Order_data = today_rows

# Fetching Price Data from another Google Sheet
SPREADSHEET = 'https://docs.google.com/spreadsheets/d/1pSHCFR-pHePTHDCMlnkS5vPS6imU_fLBk_r9eYmBuV0/edit?usp=sharing'
sht = gc.open_by_url(SPREADSHEET)
wks = sht.worksheet_by_title("工作表1")
Price_data = pd.DataFrame(wks.get_all_records())

# Cleaning and Processing Price Data
Price_data["銷售價格"] = Price_data["銷售價格"].astype(str)  # 將列轉換為字符串
# 使用正則表達式去除非數字字符
Price_data["銷售價格"] = Price_data["銷售價格"].apply(lambda x: re.sub(r'\D', '', x)).astype(int)
Price_data["商品全稱"] = Price_data["供應商"] + Price_data["商品名稱"]

# Data Processing
product_names = Price_data['商品全稱']
for index, row in Order_data.iterrows():
    subtotal = 0
    for product in product_names:
        quantity = row.get(product, 0)
        if quantity == '':
            quantity = 0
        else:
            quantity = float(quantity)
        price = float(Price_data.loc[Price_data['商品全稱'] == product, '銷售價格'].values[0])
        subtotal += quantity * price
    Order_data.at[index, '小計'] = subtotal
    Order_data = Order_data[['訂單編號', '小計'] + [col for col in Order_data.columns if col not in ['訂單編號', '小計']]]

# Here you can add code to send the bill to the corresponding email address
for index, row in Order_data.iterrows():
    email = row['電子郵件地址']
    name = row['訂購者姓名']
    subtotal = row['小計']



# 設定寄件者的電子郵件地址和密碼作為環境變數
os.environ["COOP_EMAIL"] = "coop4s.hsinchu@gmail.com"
os.environ["COOP_PASSWORD"] = "coop4s@nthu"

# Get sender email and password from environment variables
sender_email = os.getenv("COOP_EMAIL")
sender_password = os.getenv("COOP_PASSWORD")

if not sender_email or not sender_password:
    print("Sender email and password not provided in environment variables.")
    exit(1)

# Define the output directory based on the current date
today = datetime.now().strftime("%Y-%m-%d")
output_directory = f'{HOME_PATH}/content/{today}/' # gild學長在麻煩您設定成為server的路徑
os.makedirs(output_directory, exist_ok=True)  # Ensure the directory is created

# Your loop for sending emails and creating Word documents
for index, row in Order_data.iterrows():
    recipient_name = row["訂購者姓名"]
    order_num = row["訂單編號"]
    product_quantities = row[6:-2]
    total_amount = row["小計"]

    # Create email content
    email_content = f"Dear {recipient_name},\n十里方圓合作社提醒您，已收到您訂購的產品詳情如下，請確認:\n\n"
    email_content += f"===================================\n"
    email_content += f"系統自動生成的訂單編號: {order_num} \n"
    email_content += f"===================================\n"

    # Define sale_price or retrieve it from Price_data
    sale_price = 0  # Initialize with a default value
    for product, quantity in product_quantities.items():
        if quantity != "":
            # Check if the product is in the "商品名稱" column in Price_data and set sale_price accordingly
            if product in Price_data["商品全稱"].values:
                sale_price = Price_data.loc[Price_data["商品全稱"] == product, "銷售價格"].values[0]
                email_content += f"{product}: {quantity} 件，該商品單價是{sale_price}元\n"

    email_content += f"===================================\n"
    email_content += f"Total amount: {total_amount} 新台幣\n\n"
    email_content += f"===================================\n"
    email_content += f"請記得攜帶手機(確認訂單編號)與提袋\n"
    email_content += f"取貨時間請參考：https://www.coop4sustainability.live/%E6%B0%B4%E6%9C%A8%E5%8A%87%E9%BB%9E \n"
    email_content += f"取貨地點：新竹市東區光復路二段101號水木餐廳十里方圓櫃位 https://goo.gl/maps/c9UXWnfc1CfC3jGeA \n"
    email_content += f"===================================\n"
    email_content += f"溫馨提示：社員您好，若之後合作社通知取貨，且根據實際訂單金額付款完成，則此次訂單成立，因此將累積相應的t-point點數到您的賬戶，t-point點數將作為合作社年度結餘分配之依據。（t-point點數與新台幣兌換比例為1：1）\n"

    # Email settings
    sender_email = os.getenv("COOP_EMAIL")  # 使用正確的環境變數名稱
    sender_password = "jwsscmoiffdyukmm"
    recipient_email = str(row["電子郵件地址"])  # 確保將電子郵件地址轉換為字符串

    subject = "【十里方圓合作社】社員訂購產品清單確認通知"
    message = MIMEMultipart()
    message["From"] = sender_email
    message["To"] = recipient_email
    message["Subject"] = subject
    message.attach(MIMEText(email_content, "plain"))

    try:
        # Connect to SMTP server
        server = smtplib.SMTP("smtp.gmail.com", 587) #需注意中間填寫是gmail
        server.starttls()
        server.login(sender_email, sender_password)

        # Send the email
        server.sendmail(sender_email, recipient_email, message.as_string())

        # Close the SMTP server
        server.quit()

        print(f"Email sent to {recipient_name}, recipient's email address: {recipient_email}")
    except Exception as e:
        print(f"Failed to send email to {recipient_name}. Error: {str(e)}")

# Inform about the successful completion of the script
print(f"All emails saved as Word documents in {output_directory}")

### Google Sheets Authentication ###

# Define the scope and credentials for Google Sheets API
scope = [
    'https://spreadsheets.google.com/feeds',
    'https://www.googleapis.com/auth/drive'
]

# Please ensure the .json file path is correct
credentials_file = f'{HOME_PATH}orderdata-copyto-parttime-5c19ba8fcba4.json'
credentials = ServiceAccountCredentials.from_json_keyfile_name(credentials_file, scope)

# Authenticate with Google Sheets API
gc = gspread.authorize(credentials)

### Google Sheets Operations ###

# Google Sheet URL
SPREADSHEET_URL = 'https://docs.google.com/spreadsheets/d/1EDtLvs_Rz_bwsew0BfZSKHBZpp5ZNDSKg3gLv_XZCbU/edit'

# Open the Google Sheet using the provided URL
sht = gc.open_by_url(SPREADSHEET_URL)

# Specify and access the worksheet by its title
worksheet_title = "當日訂購紀錄"
wks = sht.worksheet(worksheet_title)
wks_records = wks.get_all_records()
df_wks_records = pd.DataFrame(wks_records)

# Specify and access the worksheet by its title
worksheet_title1 = "待／未完成備貨"
wks_to = sht.worksheet(worksheet_title1)
wks_to_records = wks_to.get_all_records()
df_wks_to_records = pd.DataFrame(wks_to_records)



try:
    # Move the first row of df_wks_records to the top of df_wks_to_records
    df_wks_to_records = pd.concat([df_wks_records.iloc[[0]], df_wks_to_records], ignore_index=True)

    # Append the rest of the rows from df_wks_records to df_wks_to_records
    df_wks_to_records = pd.concat([df_wks_to_records, df_wks_records.iloc[1:]], ignore_index=True)
    # 准备数据，并将无效的浮点数值替换为 None 或其他有效值
    df_wks_to_records = df_wks_to_records.where(pd.notna(df_wks_to_records), None)
    wks_to.update([df_wks_to_records.columns.values.tolist()] + df_wks_to_records.values.tolist())
except IndexError:
    # Handle the out-of-bounds index error here
    print("Index is out of bounds or the DataFrame is empty.")




# Optional: Clear the contents of the worksheet
wks.clear()

# Assuming you have a DataFrame named 'Order_data'
# Write the DataFrame contents to the Google Sheet
#set_with_dataframe(wks, Order_data)

# Assuming you have a DataFrame named 'Order_data'
# Convert the DataFrame to a list of lists
data_to_write = [Order_data.columns.tolist()] + Order_data.values.tolist()

# Update the Google Sheet with the data using the new method signature
#wks.update(values=data_to_write)

# 使用 .apply() 方法，將 '時間戳記' 欄位中的每個時間戳記轉換為指定格式的字串
Order_data['時間戳記'] = Order_data['時間戳記'].apply(convert_to_string)

# 將資料框的列轉換為二維列表，並在前面加入欄位名稱列表，然後上傳至工作表（假設使用的是 wks.update() 方法）
wks.update([Order_data.columns.values.tolist()] + Order_data.values.tolist())



Email sent to 測試陳皆錫, recipient's email address: ubo11185@gapp.nthu.edu.tw
All emails saved as Word documents in /content/2023-10-22/
Index is out of bounds or the DataFrame is empty.


{'spreadsheetId': '1EDtLvs_Rz_bwsew0BfZSKHBZpp5ZNDSKg3gLv_XZCbU',
 'updatedRange': "'當日訂購紀錄'!A1:BJ2",
 'updatedRows': 2,
 'updatedColumns': 62,
 'updatedCells': 124}

# 原始程式

In [ ]:
!pip3 install pygsheets
!pip3 install gspread-dataframe==3.2.1

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 3108, in _dep_map
    return self.__dep_map
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 2901, in __getattr__
    raise AttributeError(attr)
AttributeError: _DistInfoDistribution__dep_map

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/base_command.py", line 169, in exc_logging_wrapper
    status = run_func(*args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/req_command.py", line 242, in wrapper
    return func(self, options, args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/commands/install.py", line 441, in run
    conflicts = self._determine_conflicts(to_install)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/commands/install.py", line 

In [ ]:
import pygsheets
gc = pygsheets.authorize(service_file='/content/orderdata-copyto-parttime-5c19ba8fcba4.json') # Gild學長後續請將.json金鑰設定在local路徑

import gspread
from gspread_dataframe import get_as_dataframe, set_with_dataframe
import pandas as pd

# google sheet 網址與認證
Order_SPREADSHEET = 'https://docs.google.com/spreadsheets/d/15jbMd0cvO2OCcgxZiQJLG1jSR6rXyXn1mnl7SCoFoTM/edit?usp=sharing'
sht = gc.open_by_url(Order_SPREADSHEET)
wks = sht.worksheet_by_title("表單回應 1")
#讀取成 df
Order_data = pd.DataFrame(wks.get_all_records())

import pandas as pd
import re


# 定义一个函数来将中文时间戳转换为适合解析的格式
def convert_timestamp(timestamp):
    # 使用正则表达式将中文时间戳中的"上午"或"下午"替换为空格，同时保留原时间
    converted_timestamp = re.sub(r'(上午|下午)', '', timestamp)
    return converted_timestamp

# 对时间戳列应用转换函数
Order_data['時間戳記'] = Order_data['時間戳記'].apply(convert_timestamp)

# 将时间戳列转换为日期格式，以便按日期分组
Order_data['日期'] = pd.to_datetime(Order_data['時間戳記']).dt.strftime('%Y%m%d')

# 使用groupby对每天的订单进行累加计数，以三位数的格式表示
Order_data['當日訂單數量'] = Order_data.groupby('日期').cumcount().add(1).apply(lambda x: f'{x:03d}')

# 生成訂單編號
Order_data['訂單編號'] = Order_data['日期'] + Order_data['當日訂單數量']

# 删除不需要的列
Order_data.drop(['日期', '當日訂單數量'], axis=1, inplace=True)


from datetime import datetime
# 獲取今天的日期
today = datetime.today().date()

# 將時間戳記列拆分為日期和時間
Order_data['時間戳記']= Order_data['時間戳記'].apply(lambda x: x.split()[0])

# 將日期字符串轉換為日期對象
Order_data['時間戳記']= pd.to_datetime(Order_data['時間戳記'], format='%Y/%m/%d')

# 挑選出日期與今天相同的行
today_rows = Order_data[Order_data['時間戳記'].dt.date == today]
Order_data=today_rows


Order_data


### 「商品」對應到「供應商」

# google sheet 網址與認證
SPREADSHEET = 'https://docs.google.com/spreadsheets/d/1pSHCFR-pHePTHDCMlnkS5vPS6imU_fLBk_r9eYmBuV0/edit?usp=sharing'
sht = gc.open_by_url(SPREADSHEET)
wks = sht.worksheet_by_title("工作表1")
#讀取成 df
Price_data = pd.DataFrame(wks.get_all_records())

import re
# Assuming pricedata is a DataFrame with your data
prices = Price_data["銷售價格"].apply(lambda x: re.sub(r'\D', '', x)).astype(int)
Price_data["銷售價格"] = prices
Price_data

Price_data["商品全稱"] = Price_data["供應商"] + Price_data["商品名稱"]
Price_data



## 數據處理
# 获取 Price_data 中的商品全称列
product_names = Price_data['商品全稱']

# 遍历 Order_data 中的每一行
for index, row in Order_data.iterrows():
    subtotal = 0  # 初始化小计为0
    for product in product_names:
        quantity = row.get(product, 0)  # 获取数量，如果没有则默认为0
        if quantity == '':
            quantity = 0
        else:
            quantity = float(quantity)  # 强制转换为浮点数
        price = float(Price_data.loc[Price_data['商品全稱'] == product, '銷售價格'].values[0])  # 强制转换为浮点数
        subtotal += quantity * price
    Order_data.at[index, '小計'] = subtotal

# 根据电子邮件地址向每个订购者发送帐单
for index, row in Order_data.iterrows():
    email = row['電子郵件地址']
    name = row['訂購者姓名']
    subtotal = row['小計']
    # 此处添加发送帐单的代码，可以将帐单信息发送到对应的电子邮件地址

OrderNum = Order_data.iloc[:, -2]

# 刪除倒數第二列
Order_data.drop(Order_data.columns[-2], axis=1, inplace=True)

# 在DataFrame中的第一列插入新的Series
Order_data.insert(0, "訂單編號", OrderNum)
Order_data

!pip install python-docx

import pandas as pd
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
import os
from docx import Document
from datetime import datetime


# 設定寄件者的電子郵件地址和密碼作為環境變數
os.environ["COOP_EMAIL"] = "coop4s.hsinchu@gmail.com"
os.environ["COOP_PASSWORD"] = "coop4s@nthu"

# Get sender email and password from environment variables
sender_email = os.getenv("COOP_EMAIL")
sender_password = os.getenv("COOP_PASSWORD")

if not sender_email or not sender_password:
    print("Sender email and password not provided in environment variables.")
    exit(1)

# Define the output directory based on the current date
today = datetime.now().strftime("%Y-%m-%d")
output_directory = f'/content/{today}/' # gild學長在麻煩您設定成為server的路徑
os.makedirs(output_directory, exist_ok=True)  # Ensure the directory is created

# Your loop for sending emails and creating Word documents
for index, row in Order_data.iterrows():
    recipient_name = row["訂購者姓名"]
    order_num = row["訂單編號"]
    product_quantities = row[6:-2]
    total_amount = row["小計"]

    # Create email content
    email_content = f"Dear {recipient_name},\n十里方圓合作社提醒您，已收到您訂購的產品詳情如下，請確認:\n\n"
    email_content += f"===================================\n"
    email_content += f"系統自動生成的訂單編號: {order_num} \n"
    email_content += f"===================================\n"

    # Define sale_price or retrieve it from Price_data
    sale_price = 0  # Initialize with a default value
    for product, quantity in product_quantities.items():
        if quantity != "":
            # Check if the product is in the "商品名稱" column in Price_data and set sale_price accordingly
            if product in Price_data["商品全稱"].values:
                sale_price = Price_data.loc[Price_data["商品全稱"] == product, "銷售價格"].values[0]
            email_content += f"{product}: {quantity} 件，該商品單價是{sale_price}元\n"

    email_content += f"===================================\n"
    email_content += f"Total amount: {total_amount} 新台幣\n\n"
    email_content += f"===================================\n"
    email_content += f"請記得攜帶手機(確認訂單編號)與提袋\n"
    email_content += f"取貨時間請參考：https://www.coop4sustainability.live/%E6%B0%B4%E6%9C%A8%E5%8A%87%E9%BB%9E \n"
    email_content += f"取貨地點：新竹市東區光復路二段101號水木餐廳十里方圓櫃位 https://goo.gl/maps/c9UXWnfc1CfC3jGeA \n"
    email_content += f"===================================\n"
    email_content += f"溫馨提示：社員您好，若之後合作社通知取貨，且根據實際訂單金額付款完成，則此次訂單成立，因此將累積相應的t-point點數到您的賬戶，t-point點數將作為合作社年度結餘分配之依據。（t-point點數與新台幣兌換比例為1：1）\n"

    # Email settings
    sender_email = os.getenv("COOP_EMAIL")  # 使用正確的環境變數名稱
    sender_password = "jwsscmoiffdyukmm"
    recipient_email = str(row["電子郵件地址"])  # 確保將電子郵件地址轉換為字符串

    subject = "【十里方圓合作社】社員訂購產品清單確認通知"
    message = MIMEMultipart()
    message["From"] = sender_email
    message["To"] = recipient_email
    message["Subject"] = subject
    message.attach(MIMEText(email_content, "plain"))

    try:
        # Connect to SMTP server
        server = smtplib.SMTP("smtp.gmail.com", 587) #需注意中間填寫是gmail
        server.starttls()
        server.login(sender_email, sender_password)

        # Send the email
        server.sendmail(sender_email, recipient_email, message.as_string())

        # Close the SMTP server
        server.quit()

        print(f"Email sent to {recipient_name}, recipient's email address: {recipient_email}")
    except Exception as e:
        print(f"Failed to send email to {recipient_name}. Error: {str(e)}")


print(f"All emails saved as Word documents in {output_directory}")


### 將「當日訂購紀錄」存入excel提供給工讀生
!pip install gspread-dataframe
import gspread
from gspread_dataframe import set_with_dataframe
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd

# Define the scope and credentials
scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']
credentials = ServiceAccountCredentials.from_json_keyfile_name('orderdata-copyto-parttime-5c19ba8fcba4.json', scope) # Gild學長這要讀取.json，麻煩要在路徑中準備.json

# Authenticate with Google Sheets API
gc = gspread.authorize(credentials)

# Google Sheet URL
SPREADSHEET_URL = 'https://docs.google.com/spreadsheets/d/1EDtLvs_Rz_bwsew0BfZSKHBZpp5ZNDSKg3gLv_XZCbU/edit'

# Open the Google Sheet
sht = gc.open_by_url(SPREADSHEET_URL)

# Specify the worksheet by title
worksheet_title = "當日訂購紀錄"
wks = sht.worksheet(worksheet_title)

# Clear the contents of the worksheet (optional)
wks.clear()

# Assuming you have a DataFrame named Order_data
# Write the DataFrame to the Google Sheet
set_with_dataframe(wks, Order_data)




KeyboardInterrupt: ignored